# Procesar máscaras de cada chips y concatenar en imágenes de tamaño original

## Importar librerías y cargar variables

In [1]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import keras

import cv2


In [5]:
# Variables
valueSize = 128
im_width = valueSize
im_height = valueSize
image_size = valueSize
border = 5

In [6]:
# Cargar modelo
model_path = 'C:\\Users\\Usuario\\Desktop\\Models\\model_RESUNET2_1000epochs_128M_12bat_black_c_bcedicevales.h5'
model = load_model(model_path, compile = False) 

In [8]:
# Leer csv con la ruta de las imágenes de test
df_valid_path =  "C:\\Users\\Usuario\\Desktop\\csvs\\csvs_train\\val_all_" + str(valueSize) + '.csv'
df_valid = pd.read_csv(df_valid_path)
df_valid['0'][0]

'C:\\Users\\Usuario\\Desktop\\TFG_DATA\\my_chips_masked\\my_chips_128_masked\\L15-1617E-1207N_6468_3360_13\\chips_128\\2018_02\\global_monthly_2018_02_mosaic_L15-1617E-1207N_6468_3360_13_0.jpg'

In [7]:
# Función para aplicar un reshape a las imágenes para poder hacer la predicción 
IMAGE_CHANNEL3 = 3
def prepare3(img_aux):
  return img_aux.reshape(-1, valueSize, valueSize, IMAGE_CHANNEL3)

## Realizar predicción de todos los chips de una carpeta de test 

Es decir de un lugar únicamente en distintos momentos de tiempo

In [ ]:
imgs_valid = df_valid['0']
to_predict = []
pred_val_r = []
n = 0
count = 0
folderPrev = ""
new_names = []

# Se elige una carpeta de test y se realizan las predicciones para todas las imágenes contenidas
# Se elige la carpeta con "count"
for name_img in tqdm(imgs_valid):
    
    # Obtener carpeta de las imagen y comprobar si es distinta a la carpeta anterior (imagen de zona distinta)
    folder = name_img.split('\\',12)[7]
    if(folder!= folderPrev):
      folderPrev=folder      
      count = count+1

    # Cuando la carpeta actual es la 5  
    if(count>5):
      # Realizar predicción de todas las imágenes de la carpeta 5 y guardarlas   
      new_names.append(name_img)
      n=n+1
      img_aux = (img_to_array(load_img(name_img))) / 255  
      to_predict.append(prepare3(img_aux))
      p = model.predict(prepare3(img_aux))
      pred_val_r.append(p)
      #break'''    

print(n)

In [ ]:
lista_csv = []
lista_names = []

# Para todas las imágenes procesadas de la carpeta anterior, crear nuevo directorio y guardar 
for i, img in enumerate(pred_val_r):
    
    # Ajustar nombre para guardar  
    splitted = new_names[i].split('\\')
    lista_names.append(splitted[-1])
    #print(splitted)
    splitted.pop(4)
    splitted.pop(4)
    splitted.pop(4)
    splitted.pop(5)
    
    splitted.insert(4,'predicted_images\\RESUNET2_1000epochs_128M_12bat_black_c_bcedicevales2')
    splitted.insert(6,'val_' + str(valueSize))
    
    #print(splitted)
    save_path_joined = '\\'.join(splitted)
    lista_csv.append(save_path_joined) # Se añade la ruta del chip actual donde se va a guardar 
    #print(save_path_joined)
    #print()
    
    splitted.pop()
    save_folder = '\\'.join(splitted)
    
    # Cambiar el shape de la imagen y umbralizar con un porcentaje del 70%
    img = img.reshape(valueSize,valueSize,1)
    img = (img > 0.7).astype(np.uint8) 
    
    # Crear nuevo directorio y guardar imagen predicha umbralizada
    os.makedirs(save_folder, exist_ok = True)
    cv2.imwrite(save_path_joined, img)

In [ ]:
# Guardar la ruta a las imágenes predichas en un csv 
df1 = pd.DataFrame(lista_csv)
print(lista_csv[0])
print(df1)

df1.to_csv('C:\\Users\\Usuario\\Desktop\\csvs_pred\\RESUNET2_1000epochs_128M_12bat_black_c_bcedicevales2.csv', index=False)

## Organizar orden de lectura de los chips 

Esto se hizo pensando en procesar varias carpetas de chips a la vez pero al ser solo una con distintos meses realmente no haría falta tanto procesamiento. 

In [ ]:
print(lista_csv)

In [ ]:
# Obtener mes,ombres y carpetas
lista_concat_o = []
lista_concat_o.append([])
folders = []
names = []

prevSplitted = lista_csv[0].split('\\')[6]
prevMonth = lista_csv[0].split('\\')[8] 
names.append(lista_csv[0].split('\\')[-1])
folders.append(prevSplitted)

In [ ]:
l = 0

# Organizar chips procesados por cada mes de las imágenes del mismo lugar 
# Para cada chip procesado
for id_ in lista_csv:
    splitted = id_.split('\\')
    
    # Si se ha cambiado de carpeta de chips 
    if(splitted[6]!= prevSplitted):
        l=l+1
        lista_concat_o.append([])
        prevSplitted = splitted[6]
        folders.append(prevSplitted)
    
    # Comprobar si corresponde al mismo mes o a otro mes diferente de la misma imagen 
    if(splitted[8] != prevMonth):
        names.append(splitted[-1]) # Guardar nombre de la imagen según el mes (sin identificador del chip) 
        prevMonth = splitted[8]    # Mes actual 

    lista_concat_o[l].append('\\'.join(splitted))  # Guardar ruta de la imagen

print(lista_concat_o[0])

In [ ]:
# Cargar imágenes chips en orden 
chips_pred_test_t = []

# Para cada conjunto de imágenes de lista_concat_o
for l in range(len(lista_concat_o)):
    # Para cada chip 
    for id_ in lista_concat_o[l]:
        # Cargar y guardar en chips_pred_test_t
        img = (img_to_array(load_img(id_)))   
        chips_pred_test_t.append(img)

## Concatenar todos los chips predichos en cada máscara de tamaño original

In [ ]:
# Concatenación de chips de tamaño 256
'''concat_pred_img_t = []
concat_names = []
cont = 0

# Para cada conjunto de imágenes de chips_pred_test_t, de 16 chips en 16 para formar las imágenes predichas de cada
# mes en una sola imagen 
for k in tqdm(range(0,len(chips_pred_test_t),16)):
    
  concat_h1_c = cv2.hconcat([chips_pred_test_t[k+0], chips_pred_test_t[k+1], chips_pred_test_t[k+2], chips_pred_test_t[k+3]])
  concat_h2_c = cv2.hconcat([chips_pred_test_t[k+4], chips_pred_test_t[k+5], chips_pred_test_t[k+6], chips_pred_test_t[k+7]])
  concat_h3_c = cv2.hconcat([chips_pred_test_t[k+8], chips_pred_test_t[k+9], chips_pred_test_t[k+10], chips_pred_test_t[k+11]])
  concat_h4_c = cv2.hconcat([chips_pred_test_t[k+12], chips_pred_test_t[k+13], chips_pred_test_t[k+14], chips_pred_test_t[k+15]])

  concat_c = cv2.vconcat([concat_h1_c, concat_h2_c, concat_h3_c, concat_h4_c])
  concat_pred_img_t.append(concat_c) 
'''

In [ ]:
# Concatenación de chips de tamaño 128
concat_pred_img_t = []
concat_names = []
cont = 0


# Para cada conjunto de imágenes de chips_pred_test_t, de 64 chips en 64 para formar las imágenes predichas de cada
# mes en una sola imagen 
for k in tqdm(range(0,len(chips_pred_test_t),64)):
    
  concat_h1_c = cv2.hconcat([chips_pred_test_t[k+0],  chips_pred_test_t[k+1],  chips_pred_test_t[k+2],  chips_pred_test_t[k+3],   chips_pred_test_t[k+4],  chips_pred_test_t[k+5],  chips_pred_test_t[k+6],  chips_pred_test_t[k+7]])
  concat_h2_c = cv2.hconcat([chips_pred_test_t[k+8],  chips_pred_test_t[k+9],  chips_pred_test_t[k+10], chips_pred_test_t[k+11], chips_pred_test_t[k+12], chips_pred_test_t[k+13], chips_pred_test_t[k+14], chips_pred_test_t[k+15]])
  concat_h3_c = cv2.hconcat([chips_pred_test_t[k+16], chips_pred_test_t[k+17], chips_pred_test_t[k+18], chips_pred_test_t[k+19], chips_pred_test_t[k+20], chips_pred_test_t[k+21], chips_pred_test_t[k+22], chips_pred_test_t[k+23]])
  concat_h4_c = cv2.hconcat([chips_pred_test_t[k+24], chips_pred_test_t[k+25], chips_pred_test_t[k+26], chips_pred_test_t[k+27], chips_pred_test_t[k+28], chips_pred_test_t[k+29], chips_pred_test_t[k+30], chips_pred_test_t[k+31]])
  concat_h5_c = cv2.hconcat([chips_pred_test_t[k+32],  chips_pred_test_t[k+33],  chips_pred_test_t[k+34],  chips_pred_test_t[k+35],   chips_pred_test_t[k+36],  chips_pred_test_t[k+37],  chips_pred_test_t[k+38],  chips_pred_test_t[k+39]])
  concat_h6_c = cv2.hconcat([chips_pred_test_t[k+40],  chips_pred_test_t[k+41],  chips_pred_test_t[k+42], chips_pred_test_t[k+43], chips_pred_test_t[k+44], chips_pred_test_t[k+45], chips_pred_test_t[k+46], chips_pred_test_t[k+47]])
  concat_h7_c = cv2.hconcat([chips_pred_test_t[k+48], chips_pred_test_t[k+49], chips_pred_test_t[k+50], chips_pred_test_t[k+51], chips_pred_test_t[k+52], chips_pred_test_t[k+53], chips_pred_test_t[k+54], chips_pred_test_t[k+55]])
  concat_h8_c = cv2.hconcat([chips_pred_test_t[k+56], chips_pred_test_t[k+57], chips_pred_test_t[k+58], chips_pred_test_t[k+59], chips_pred_test_t[k+60], chips_pred_test_t[k+61], chips_pred_test_t[k+62], chips_pred_test_t[k+63]])

    
  concat_c = cv2.vconcat([concat_h1_c, concat_h2_c, concat_h3_c, concat_h4_c, concat_h5_c, concat_h6_c, concat_h7_c, concat_h8_c])
  concat_pred_img_t.append(concat_c) 

In [ ]:
#plt.figure(figsize= (10,10))
#plt.imshow(concat_pred_img_t[0].squeeze(), vmin=0, vmax=1)

## Guardar concatenaciones y rutas

In [ ]:
# Guardar los chips concatenados en las imágenes de tamaño original 
lista_concat_save = []
save_path = "C:\\Users\\Usuario\\Desktop\\predicted_images\\concat_RESUNET2_1000epochs_128M_12bat_black_c_bcedicevales2\\"
print(names)
for i, name_img in enumerate(names):
    
    save_path_joined = save_path + name_img    
    lista_concat_save.append(save_path_joined)
        
    os.makedirs(save_path, exist_ok = True)
    cv2.imwrite(save_path_joined, concat_pred_img_t[i])

In [ ]:
# Guardar csv con la ruta a las imágenes concatenadas
df1 = pd.DataFrame(lista_concat_save)
#df2 = pd.DataFrame({'imgs': X_valid, 'masks': y_valid})
#print(df1)
#print(df2)
#print(n)

df1.to_csv('C:\\Users\\Usuario\\Desktop\\csvs_pred\\concat_model_RESUNET2_1000epochs_128M_12bat_black_c_bcedicevales2.csv', index=False)
#df2.to_csv('C:\\Users\\Usuario\\Desktop\\val_512.csv', index=False)